### 웹페이지 접근

In [98]:
from selenium import webdriver
import time

# 크롬 웹드라이버 객체 생성
driver = webdriver.Chrome()

# 원하는 페이지로 이동하기
url = "https://finance.naver.com/news/mainnews.naver"
driver.get(url)
time.sleep(1)

# 브라우저 크기 최대화
driver.maximize_window()
time.sleep(1)

# 현재 페이지의 HTML 소스코드 가져오기
html = driver.page_source

### HTML 파싱

In [99]:
from bs4 import BeautifulSoup

# HTML 문자열을 파싱하여 BeautifulSoup 객체 생성
# 이를 통해 HTML 구조를 탐색하고 데이터를 추출할 수 있음
soup = BeautifulSoup(html, 'html.parser')

# soup : 파싱된 HTML을 담는 BeautifulSoup 객체로, 이를 통해 데이터를 쉽게 추출할 수 있음
# html : webdriver에서 받아온 html 소스코드
# 'html.parser': BeautifulSoup이 사용할 파서(parser) 지정

### 데이터 추출

In [100]:
# 첫번째 아티클의 제목 추출
title = soup.select_one('.articleSubject').text.strip('\n')
print(title)

"테슬라 반등했는데 한국은 언제쯤"…개미들 속탄다 [한경우의 케이스스터디]


In [41]:
# 첫번째 아티클의 링크 추출
soup.select_one('.articleSubject>a').attrs["href"]

'https://n.news.naver.com/mnews/article/421/0007646791'

In [83]:
# 모든 아티클 목록으로 반복하기
articles = soup.select('.block1')
article_list = []
for article in articles:
    # 제목, 상세페이지url, 내용, 언론사, 날짜 가져오기
    title = article.select_one(".articleSubject").text.strip()
    url = article.select_one(".articleSubject>a").attrs["href"]
    summary = article.select_one(".articleSummary").contents[0].strip()
    press = article.select_one(".press").text
    date = article.select_one(".wdate").text
    article_list.append([title, url, summary, press, date])

    

[["'10만전자' 다시 오나…'52주 최고가' 삼성전자, 주가 향방은[종목현미경]", 'https://n.news.naver.com/mnews/article/421/0007646791', "삼성전자(005930)가 2분기 '어닝 서프라이즈'에 힘입어 3년 5개월 만에 최고가를 기록했다. 특히 개인 투자자들의 매물이 3년 넘..", '뉴스1 ', '2024-07-06 00:54:53'], ["뉴욕증시, '냉온탕' 비농업 고용 지표에 혼조 출발", 'https://n.news.naver.com/mnews/article/001/0014790810', '진정호 연합인포맥스 특파원 = 뉴욕증시가 미국 6월 비농업 부문 고용 결과를 받아본 뒤 혼조 양상을 보이고 있다. 고용은 예상치를 웃돌..', '연합뉴스 ', '2024-07-06 00:14:35'], ['MBK파트너스, 헬스케어 힘주나…日 제약사에 3조 베팅 이유는[주간사모펀드]', 'https://n.news.naver.com/mnews/article/629/0000302170', 'JKL파트너스, 티웨이항공 엑시트 추진 공무원연금공단, MBK파트너스·IMM PE 등 사모대체 블라인드 펀드 위탁운용사 선정 국내 사모..', '더팩트 ', '2024-07-06 00:01:48']]


### 데이터 저장

In [86]:
import pandas as pd

# 데이터프레임으로 만들기
df = pd.DataFrame(article_list,
             columns = ['title','url','summary','press','date'])

# csv 파일로 저장하기
df.to_csv('20240706_네이버증권뉴스.csv', index=False)

# 저장한 csv파일을 읽어와 확인하기
pd.read_csv('20240706_네이버증권뉴스.csv')

### 코드 합치기

In [101]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 뉴스기사 날짜 입력받기
input_date = input('검색할 날짜(yyyy-nn-dd)-->')

# 웹드라이버 생성
driver = webdriver.Chrome()

article_list = []
for page in range(1,4):
    time.sleep(2)
    # 웹페이지 접근
    url = f"https://finance.naver.com/news/mainnews.naver?date={input_date}&page={page}"
    driver.get(url)
    time.sleep(1)
    driver.maximize_window()
    time.sleep(1)

    # HTML 문자열을 파싱하여 BeautifulSoup 객체 생성
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 모든 아티클 리스트 생성하기
    articles = soup.select('.block1')    
    for article in articles:
        # 제목, 상세페이지url, 내용, 언론사, 날짜 가져오기
        title = article.select_one(".articleSubject").text.strip()
        url = article.select_one(".articleSubject>a").attrs["href"]
        summary = article.select_one(".articleSummary").contents[0].strip()
        press = article.select_one(".press").text
        date = article.select_one(".wdate").text
        article_list.append([title, url, summary, press, date])

# 파일로 저장하기
df = pd.DataFrame(article_list,
             columns = ['title','url','summary','press','date'])
df.to_csv('20240706_네이버증권뉴스.csv', index=False)
# pd.read_csv('20240706_네이버증권뉴스.csv')

### 마지막 페이지까지 크롤링

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 뉴스기사 날짜 입력받기
input_date = input('검색할 날짜(yyyy-nn-dd)-->')

# 웹드라이버 생성
driver = webdriver.Chrome()

article_list = []
page = 1
while True:
    time.sleep(2)
    # 웹페이지 접근
    url = f"https://finance.naver.com/news/mainnews.naver?date={input_date}&page={page}"
    driver.get(url)
    time.sleep(1)
    driver.maximize_window()
    time.sleep(1)

    # HTML 문자열을 파싱하여 BeautifulSoup 객체 생성
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 모든 아티클 리스트 생성하기
    articles = soup.select('.block1')    
    for article in articles:
        # 제목, 상세페이지url, 내용, 언론사, 날짜 가져오기
        title = article.select_one(".articleSubject").text.strip()
        url = article.select_one(".articleSubject>a").attrs["href"]
        summary = article.select_one(".articleSummary").contents[0].strip()
        press = article.select_one(".press").text
        date = article.select_one(".wdate").text
        article_list.append([title, url, summary, press, date])
    
    # pgRR 요소가 있으면 page에 1 더하기, 없으면 반복 중단
    if not soup.select_one('.pgRR'): break
    else: page+=1


# 파일로 저장하기
file_name = f'네이버증권뉴스_{input_date.replace('-','')}' # 파일명 생성

df = pd.DataFrame(article_list,
             columns = ['title','url','summary','press','date']) # 데이터프레임 생성

df.to_csv(file_name+'.csv', index=False) # csv파일로 저장

import openpyxl 
df.to_excel(file_name+'.xlsx', index=False) # excel 파일로 저장

driver.quit() # 웹드라이브 종료